In [51]:
from GMMHMM import *
import numpy as np

In [52]:
import pickle
def load_pkl_file(foldername, filename):
    filepath=foldername+'/'+filename+'.pkl'
    file=open(filepath,'rb')
    hmm_model=pickle.load(file)
    return hmm_model

def load_hmm_models(foldername,filenames):
    HMM_models={}
    for num in filenames:
        hmm_model=load_pkl_file(foldername,str(num))
        HMM_models[str(num)]=hmm_model
    return HMM_models

In [53]:
hmm_models=load_hmm_models('hmm_model',[0,1,2,3,4,5,6,7,8,9])

In [86]:
class Node:
    def __init__(self,val, word):
        self.val=val
        self.next=[]
        self.word=word
        #three states: -1: root, 1: end, 0: others
        self.state=0        

In [96]:
class LexicalTree:
    def __init__(self,models):
        self.getwords(models)
        zeros=np.zeros([39])
        ones=np.array([1 for i in range(39)])
        # set the initial state as a Gaussian mixture model
        initial_GMM=mixInfo()
        initial_GMM.mean.append(zeros)
        initial_GMM.Gaussian_var.append(ones)
        initial_GMM.mean=np.array(initial_GMM.mean)
        initial_GMM.Gaussian_var=np.array(initial_GMM.Gaussian_var)
        initial_GMM.Gaussian_weight=[1]
        initial_GMM.Num_of_Gaussian=1
        # let the initial state as the root of the lexical tree
        self.root=Node(initial_GMM,'*')
        self.root.state=-1
           
    # get model and transition cost for each digits
    def getwords(self,models):
        self.words=[]
        self.digits=list(models.keys())
        self.edge_cost={}
        for digit in self.digits:
            self.words.append(models[digit])
            self.edge_cost[digit]=models[digit].hmm.transition_cost

    def GenerateTree(self):
        for i in range(len(self.words)):
            word=self.words[i]
            digit=self.digits[i]
            # the initial state for ith digit
            previous=Node(word.hmm.mix[0],digit)
            # which is one of the next option for the root node
            self.root.next.append(previous)
            # other states for the digit
            for j in range(1,word.hmm.N):
                curr=Node(word.hmm.mix[j],digit)
                previous.next.append(curr)
                previous=curr
            # for the last state for the ith digit, its next node is root
            previous.next.append(self.root)
            previous.state=1 

In [101]:
lextree=LexicalTree(hmm_models)
lextree.GenerateTree()
root=lextree.root
edge_cost=lextree.edge_cost

In [108]:
class ContinuousSpeechRecognition():
    def __init__(self):
        self.lextree=None
        
    def get_tree_info(self,root,edge_cost):
        self.lextree=root
        self.get_nodes(self.lextree)
        self.edge_cost=edge_cost
        self.previous={}
        self.next={}
        self.end_nodes=[]
        for i in range(len(self.nodes)):
            node=self.nodes[i]
            if node.state==1:
                self.end_nodes.append(i)
            self.next[i]=[]
            if len(node.next)>0:
                for next_node in node.next:
                    self.next[i].append(self.nodes.index(next_node))
                    self.previous[self.nodes.index(next_node)]=i

    def get_nodes(self,root):
        self.nodes=[root]
        self.init_nodes=[]
        self.states=[0]
        for node in root.next:
            state=0
            curr_node=node
            # add the first node of each word into init_nodes list
            self.init_nodes.append(node)
            # while the node is not the ending node of a word(digit)
            while curr_node.state!=1:
                state+=1
                self.nodes.append(curr_node)
                self.states.append(state)
                curr_node=curr_node.next[0]
            # add the ending node of a word to the nodes list
            state+=1
            self.nodes.append(curr_node)
            self.states.append(state)

    def digit_recognition_47(self,data,cost):
        zeros=np.zeros(np.shape(data)[1])
        data=np.vstack([zeros,data])
        rows=len(self.nodes)
        cols=len(self.data)
        
        all_costs=np.full((rows,cols),np.inf)
        all_costs[0][0]=0
        all_costs=[all_costs]
        # track the current column
        curr_col=[np.zeros(rows)]
        for i in range(1,cols):
            next_col=[]
            for j in range(len(all_costs)):
                curr_costs=all_costs[j]
                curr_nodes=curr_col[j]
                next_nodes=np.zeros(rows)
                for k in range(1,rows):
                    # calculate the mixture log gaussian score of the nodes and the vector in input data
                    score=mixture_log_gaussian(self.nodes[k].val,data[i])
                    if curr_nodes[k]:

    def 

In [109]:
csr=ContinuousSpeechRecognition()
csr.get_tree_info(root,edge_cost)